In [2]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from datetime import datetime
from ultralytics import YOLO
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, List


@dataclass
class PoseConfig:
    model_weights: str = 'yolov8x-pose.pt'
    threshold: float = 0.5
    iou: float = 0.7
    device: str = 'cuda'
    use_smoothing: bool = True
    smoothing_window: int = 5


class SwimmerPoseAnalyzer:
    def __init__(self, config: PoseConfig):
        self.config = config
        self.model = YOLO(config.model_weights)
        self.model.conf = config.threshold
        self.model.iou = config.iou
        self.keypoint_cache = {}
        self.limb_pairs = [
            (5, 6), (5, 11), (6, 12), (11, 12),
            (5, 7), (7, 9), (6, 8), (8, 10),
            (11, 13), (13, 15), (12, 14), (14, 16)
        ]

    def _apply_smoothing(self, keypoints: np.ndarray, idx: int) -> np.ndarray:
        if not self.config.use_smoothing:
            return keypoints

        self.keypoint_cache[idx] = keypoints

        recent_frames = [self.keypoint_cache[i] for i in range(
            max(0, idx - self.config.smoothing_window), idx + 1
        ) if i in self.keypoint_cache]

        weights = np.exp(np.linspace(-1, 0, len(recent_frames)))
        weights /= weights.sum()

        smoothed = sum(w * kp for w, kp in zip(weights, recent_frames))
        # Limit memory
        for i in list(self.keypoint_cache.keys()):
            if i < idx - self.config.smoothing_window:
                del self.keypoint_cache[i]

        return smoothed

    def _draw_pose(self, image: np.ndarray, keypoints: np.ndarray, score: float, mode='full') -> np.ndarray:
        output = image.copy()

        if mode in ['full', 'stick']:
            for pt1_idx, pt2_idx in self.limb_pairs:
                x1, y1 = keypoints[pt1_idx][:2]
                x2, y2 = keypoints[pt2_idx][:2]
                if all(0 <= val < dim for val, dim in zip((x1, y1, x2, y2), image.shape[1::-1] * 2)):
                    cv2.line(output, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        if mode in ['full', 'points']:
            for x, y, c in keypoints:
                if c > self.config.threshold:
                    cv2.circle(output, (int(x), int(y)), 4, (0, 0, 255), -1)

        cv2.putText(output, f"Conf: {score:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        return output

    def process_single_image(self, path: str, save_dir: str, index: int) -> Tuple[bool, Optional[Dict]]:
        img = cv2.imread(path)
        if img is None:
            return False, None

        try:
            result = self.model(img, verbose=False)
            if not result or not result[0].keypoints:
                return False, None

            keypoints = result[0].keypoints[0].data[0].cpu().numpy()
            confidence = float(result[0].boxes[0].conf)
            keypoints = self._apply_smoothing(keypoints, index)

            ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            filename = os.path.splitext(os.path.basename(path))[0]

            for view in ['full', 'stick', 'points']:
                vis = self._draw_pose(img, keypoints, confidence, view)
                out_path = os.path.join(save_dir, f"pose_{view}_{ts}_{filename}.jpg")
                cv2.imwrite(out_path, vis)

            return True, {
                'keypoints': keypoints.tolist(),
                'confidence': confidence,
                'frame_id': index
            }

        except Exception as e:
            print(f"[!] Failed processing {path}: {e}")
            return False, None

    def run_batch_estimation(self, source_folder: str, output_folder: str):
        os.makedirs(output_folder, exist_ok=True)
        images = sorted([
            f for f in os.listdir(source_folder)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])

        if not images:
            print(" No images found.")
            return

        results = []
        total, successes, failures = len(images), 0, 0

        print(f"\n🧠 Starting batch pose estimation on {total} images...")
        for i, img_name in enumerate(tqdm(images)):
            img_path = os.path.join(source_folder, img_name)
            ok, data = self.process_single_image(img_path, output_folder, i)

            if ok:
                successes += 1
                results.append({
                    'filename': img_name,
                    'pose_data': data
                })
            else:
                failures += 1

        if results:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            np.save(os.path.join(output_folder, f"pose_data_{timestamp}.npy"), results)

        print(f"\n Finished:")
        print(f"Total: {total}")
        print(f"Detected: {successes}")
        print(f"Errors: {failures}")
        print(f"Results saved to: {output_folder}")


# --- Run This Script ---
if __name__ == "__main__":
    try:
        CONFIG = PoseConfig(
            model_weights='yolov8x-pose.pt',
            threshold=0.5,
            iou=0.7,
            device='cuda',
            use_smoothing=True,
            smoothing_window=5
        )

        estimator = SwimmerPoseAnalyzer(CONFIG)

        INPUT_DIR = r"C:\Users\Jay\Desktop\CV_Project\Swimmer_Detections"
        OUTPUT_DIR = r"C:\Users\Jay\Desktop\CV_Project\Swimmer_yolo_estimation"

        estimator.run_batch_estimation(INPUT_DIR, OUTPUT_DIR)

    except Exception as err:
        print(f"\n Runtime error: {err}")
    finally:
        print("\n Pose estimation process completed.")



00%|███████████████████████████████████████████████████████████████████████████████| 133M/133M [00:03<00:00, 44.3MB/s]


🧠 Starting batch pose estimation on 2556 images...


  2%|█▊                                                                              | 59/2556 [00:12<05:02,  8.26it/s]

[!] Failed processing C:\Users\Jay\Desktop\CV_Project\Swimmer_Detections\detection_20250509_215753_693088_frame_001095_20250509_032723_093252.jpg: index 0 is out of bounds for dimension 0 with size 0


  3%|██                                                                              | 65/2556 [00:13<04:19,  9.60it/s]

...

[!] Failed processing C:\Users\Jay\Desktop\CV_Project\Swimmer_Detections\swimmer_20250509_221919_308516_frame_045588_20250509_035042_419091_0.jpg: index 0 is out of bounds for dimension 0 with size 0
[!] Failed processing C:\Users\Jay\Desktop\CV_Project\Swimmer_Detections\swimmer_20250509_221925_673598_frame_045981_20250509_035055_669977_0.jpg: index 0 is out of bounds for dimension 0 with size 0


100%|█████████████████████████████████████████████████████████████████████████████▊| 2551/2556 [04:30<00:00, 13.91it/s]

[!] Failed processing C:\Users\Jay\Desktop\CV_Project\Swimmer_Detections\swimmer_20250509_221930_499626_frame_048117_20250509_035146_764796_0.jpg: index 0 is out of bounds for dimension 0 with size 0


100%|██████████████████████████████████████████████████████████████████████████████| 2556/2556 [04:30<00:00,  9.44it/s]

[!] Failed processing C:\Users\Jay\Desktop\CV_Project\Swimmer_Detections\swimmer_20250509_221942_453405_frame_049647_20250509_035225_743936_0.jpg: index 0 is out of bounds for dimension 0 with size 0

✅ Finished:
🔢 Total: 2556
✅ Detected: 2350
❌ Errors: 206
📂 Results saved to: C:\Users\Jay\Desktop\CV_Project\Swimmer_yolo_estimation

🏁 Pose estimation process completed.
